In [1]:
import pandas as pd
import json
from datetime import datetime

In [ ]:
data = pd.read_csv('PSB/data/egrul.csv')

,id,inn,data,fetched_at
0,1,1001028755,"{""data"": {""ИНН"": ""1001028755"", ""КПП"": ""7806010...",2025-04-21 13:02:51.38553
1,2,1001001390,"{""data"": {""ИНН"": ""1001001390"", ""КПП"": ""1001010...",2025-04-21 13:02:51.398212
2,3,1001098784,"{""data"": {""ИНН"": ""1001098784"", ""КПП"": ""1001010...",2025-04-21 13:02:51.431791
3,4,1001095007,"{""data"": {""ИНН"": ""1001095007"", ""КПП"": ""1001010...",2025-04-21 13:02:51.439663
4,5,1001004095,"{""data"": {""ИНН"": ""1001004095"", ""КПП"": ""1001010...",2025-04-21 13:02:51.489868


In [3]:
data['data'] = data['data'].apply(json.loads)

In [ ]:
df_expanded = pd.json_normalize(data['data'])
df_expanded.columns = [col.replace('.', '_') for col in df_expanded.columns]

,data_ИНН,data_КПП,data_СЧР,data_ОГРН,data_ОКПО,data_ОКФС_Код,data_ОКФС_Наим,data_РМСП_Кат,data_РМСП_ДатаВкл,data_ЕФРСБ,...,data_Ликвид_Дата,data_Ликвид_Наим,data_Контакты_Телеграм,data_Подразд_Филиал,data_Подразд_Представ,data_СанкцииСтраны,data_НедобПостЗап,data_ЮрАдрес_НедостОпис,data_ЮрАдрес_НедостДатаЗаписи,data_НелегалФинСтатус
0,1001028755,780601001,39,1111001006698,67882274,16,Частная собственность,МАЛОЕ ПРЕДПРИЯТИЕ,2016-08-01,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001001390,100101001,163,1021000513984,00245552,42,Смешанная российская собственность с долей соб...,NaN,NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001098784,100101001,4,1021000507340,53202755,16,Частная собственность,МАЛОЕ ПРЕДПРИЯТИЕ,2022-01-10,"[{'Тип': 'Сведения о судебном акте', 'Дата': '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1001095007,100101001,3,1021000507945,51326016,16,Частная собственность,МАЛОЕ ПРЕДПРИЯТИЕ,2016-08-01,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1001004095,100101001,37,1021000534752,16822388,16,Частная собственность,МАЛОЕ ПРЕДПРИЯТИЕ,2021-09-10,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
features = pd.DataFrame()
features['inn'] = df_expanded['data_ИНН']

In [7]:
features['categories'] = df_expanded['data_РМСП_Кат']

In [11]:
# 1. Возраст компании (в годах)
df_expanded['ДатаРег'] = pd.to_datetime(df_expanded['data_ДатаРег'], errors='coerce')
df_expanded['Возраст_компании'] = ((datetime.now() - df_expanded['ДатаРег']).dt.days // 365).fillna(-1)

# 2. Статус компании
df_expanded['Статус'] = df_expanded['data_Статус_Наим'].fillna('data_Неизвестно')

# 3. Недостоверный юридический адрес
df_expanded['Недостоверный_адрес'] = df_expanded['data_ЮрАдрес_Недост'].fillna(False)

# 4. Сумма налоговой недоимки
df_expanded['Сумма_налоговой_недоимки'] = df_expanded['data_Налоги_СумНедоим'].fillna(0).astype(float)

# 5. Сумма уплаченных налогов
df_expanded['Сумма_уплаченных_налогов'] = df_expanded['data_Налоги_СумУпл'].fillna(0).astype(float)

# 6. Применение особого налогового режима
df_expanded['Особый_налоговый_режим'] = df_expanded['data_Налоги_ОсобРежим'].apply(lambda x: 1 if isinstance(x, list) and len(x) > 0 else 0)

# 7. Массовый руководитель
df_expanded['Массовый_руководитель'] = df_expanded['data_МассРуковод'].fillna(False)

# 8. Дисквалифицированный руководитель
df_expanded['Дисквалифицированный_руководитель'] = df_expanded['data_ДисквЛица'].fillna(False)

# 9. Недостоверные сведения о руководстве
df_expanded['Недостоверный_руководитель'] = df_expanded['data_Руковод'].apply(
    lambda x: any(d.get('Недост', False) for d in x) if isinstance(x, list) else False
)

# 10. Массовые учредители
df_expanded['Массовый_учредитель'] = df_expanded['data_МассУчред'].fillna(False)

# 11. Наличие записей о банкротстве
df_expanded['Есть_банкротство'] = df_expanded['data_ЕФРСБ'].apply(lambda x: 1 if isinstance(x, list) and len(x) > 0 else 0)

# 12. Включение в реестр недобросовестных поставщиков
df_expanded['Недобросовестный_поставщик'] = df_expanded['data_НедобПост'].fillna(False)

# 13. Наличие санкций
df_expanded['Под_санкциями'] = df_expanded['data_Санкции'].fillna(False)

# 14. Категория предприятия
df_expanded['Категория_МСП'] = df_expanded['data_РМСП_Кат'].fillna('Неизвестно')

# 15. Среднесписочная численность работников
df_expanded['Среднесписочная_численность'] = df_expanded['data_СЧР'].fillna(0)

df_expanded = df_expanded[[
    "data_ИНН", 
    "data_ОКАТО_Наим", 
    "data_ОКВЭД_Код", 
    "data_ОКВЭД_Наим", 
    "data_ОКОГУ_Наим", 
    "data_Налоги_СумУпл", 
    "data_Налоги_СумНедоим", 
    "data_УстКап_Сумма",
    "data_Санкции", 
    "data_НаимПолн", 
    "data_НелегалФин", 
    "Возраст_компании", 
    "Статус", 
    "Недостоверный_адрес", 
    "Особый_налоговый_режим", 
    "Массовый_учредитель", 
    "Есть_банкротство", 
    "Недобросовестный_поставщик", 
    "Категория_МСП", 
    "Среднесписочная_численность"
]]

df_expanded.to_csv('PSB/data/egrul_processed.csv', index=False)

,data_ИНН,data_ОКАТО_Наим,data_ОКВЭД_Код,data_ОКВЭД_Наим,data_ОКОГУ_Наим,data_Налоги_СумУпл,data_Налоги_СумНедоим,data_УстКап_Сумма,data_Санкции,data_НаимПолн,data_НелегалФин,Возраст_компании,Статус,Недостоверный_адрес,Особый_налоговый_режим,Массовый_учредитель,Есть_банкротство,Недобросовестный_поставщик,Категория_МСП,Среднесписочная_численность
0,1001028755,Красногвардейский,46.73,"Торговля оптовая лесоматериалами, строительным...","Организации, учрежденные юридическими лицами и...",4668452.57,None,10000.0,False,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЭЛЕМ...",False,13,Действует,False,0,True,0,False,МАЛОЕ ПРЕДПРИЯТИЕ,39
1,1001001390,Петрозаводск,02.20,Лесозаготовки,"Хозяйственные общества, образованные из госуда...",247854589.77,20723099.17,15583520.0,False,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ЛЕСОПРОМЫШЛЕННАЯ ХОЛДИНГ...",False,30,Действует,False,0,False,0,False,Неизвестно,163
2,1001098784,Петрозаводск,10.52,Производство мороженого,"Организации, учрежденные юридическими лицами и...",13317513.78,126958386.04,9850000.0,False,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ХОЛОД СЛАВМО""",False,25,Юридическое лицо признано несостоятельным (бан...,False,0,False,1,False,МАЛОЕ ПРЕДПРИЯТИЕ,4
3,1001095007,Петрозаводск,46.51,"Торговля оптовая компьютерами, периферийными у...","Организации, учрежденные юридическими лицами и...",5535988.91,None,210000.0,False,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""НОУТ...",False,25,Действует,False,0,False,0,False,МАЛОЕ ПРЕДПРИЯТИЕ,3
4,1001004095,Петрозаводск,20.11,Производство промышленных газов,"Организации, учрежденные юридическими лицами и...",16959388.26,None,32400.0,False,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АКАР""",False,32,Действует,False,0,False,0,False,МАЛОЕ ПРЕДПРИЯТИЕ,37
5,1001146519,Петрозаводск,46.73.1,Торговля оптовая древесным сырьем и необработа...,"Организации, учрежденные юридическими лицами и...",7087215.53,None,1342100.0,False,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СП ""...",False,21,Действует,False,0,False,0,False,МАЛОЕ ПРЕДПРИЯТИЕ,1
6,1001174611,Петрозаводск,16.10,Распиловка и строгание древесины,"Организации, учрежденные юридическими лицами и...",3456069.90,None,10000.0,False,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""НОРД...",False,19,Действует,False,1,False,0,False,МИКРОПРЕДПРИЯТИЕ,7
7,1001160954,Петрозаводск,22.23,"Производство пластмассовых изделий, используем...","Организации, учрежденные юридическими лицами и...",12704070.38,None,10000.0,False,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ЗАВОД...,False,20,Не действует,False,0,False,0,False,Неизвестно,10
8,1001218280,Петрозаводск,46.66,Торговля оптовая прочей офисной техникой и обо...,"Организации, учрежденные юридическими лицами и...",1754298.13,None,10000.0,False,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СМС""",False,16,Действует,False,1,False,0,False,МИКРОПРЕДПРИЯТИЕ,5
9,1001229243,Петрозаводск,45.11.2,Торговля розничная легковыми автомобилями и ле...,"Организации, учрежденные юридическими лицами и...",17044946.55,None,9000000.0,False,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СЕВЕ...",False,15,Не действует,False,0,False,0,False,Неизвестно,26
